# 1.Data Audit & 2.Framework Model

In [ ]:
# Support modules

import glob, os, re, random
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.stats as st
import lasio # Las file reader module
from datetime import datetime
from difflib import SequenceMatcher
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import curve_fit

## Import data

In [ ]:
note = """
1.) This is for modeling based on isotropic homogeneous material without overpressure assumption using statistic decisions or machine learning techniques. 

2.) The working directory should contain the data for modeling as sub-directory. 

3.) All data for modeling include well logging files (.las), deviation files (.csv) and formation top (.csv) must be separated
as sub-directory of the data directory. 
For example;
- Working directory is "Drive:/Working/".
- All data for modeling directory is "Drive:/Working/Data/".
- Well logging file directory is "Drive:/Working/Data/Well logging/" as Sub-directory of the data directory.
- Deviation file directory is "Drive:/Working/Data/Deviation/" as Sub-directory of the data directory.
- Formation top file directory is "Drive:/Working/Data/Formation top/" as Sub-directory of the data directory.

4.) Well name should be set as prefix for each file. Its name will cause file ordering and file pairing for each file of that well.
For example;
- Well name is "Well-01" (Noted: No underscore ('_') be contained in well name), so this name should be set as prefix followed by underscore ('_') for each modeling input file like this "Well-01_(...Specific name for file type indication...)"
- Example; Well logging file name, Deviation file name and Formation top file name could be "Well-01_las", "Well-01_dev" and "Well-01_top" respectively.

5.: Required data and file format;
- Well logging files include all necessary curves for 1D MEM such caliper (CAL), bitsize (BS), gamma ray (GR), density (RHOB), neutron porosity (NPHI), deep resistivity (RT), shallow resistivity (MSFL), P-sonic (DTC) and S-sonic (DTS).
- Deviation files include measured depth column named with 'MD', azimuth column named with 'AZIMUTH' and inclination or angle column named with 'ANGLE'.
- Formation top files include formation name column named with 'Formations', top depth column named with 'Top' and bottom depth column named with 'Bottom'.
"""
print('Welcome to Automated 1D Mechanical Earth Modeling (Auto 1D MEM).')
print('Please take note on this;')
print(note)

In [ ]:
# Setup data directory

cwd_dir_list = ', '.join(os.listdir(os.getcwd()))
confirm = 'no'

print('According to your working directory,')
print('%s\nwhich one is your data directory?' %cwd_dir_list)

while confirm.lower() == 'no':
    data_folder = input('Please indicate the data directory name: ').strip()
    data_path = os.path.join(os.getcwd(), data_folder)

    if data_folder == '':
        print('Please type the directory name!')
        continue

    elif os.path.isdir(data_path):
        data_dir_list = ', '.join(os.listdir(data_path))
        print('%s\nThese sub-directories are found.' %data_dir_list)

        while True:
            print('Which one is your Well logging file directory?')
            las_folder = input('Please indicate the well logging file directory name (.las): ').strip()
            las_path = os.path.join(os.getcwd(), data_folder, las_folder)

            if las_folder == '':
                print('Please type the directory name!')
                continue

            elif os.path.isdir(las_path):
                
                while True:
                    print('Which one is your deviation file directory?')
                    dev_folder = input('Please indicate the deviation file directory name (.csv): ').strip()
                    dev_path = os.path.join(os.getcwd(), data_folder, dev_folder)

                    if dev_folder == '':
                        print('Please type the directory name!')
                        continue

                    elif os.path.isdir(dev_path):

                        while True:
                            print('Which one is your formation top file directory?')
                            top_folder = input('Please indicate the formation top file directory name (.csv): ').strip()
                            top_path = os.path.join(os.getcwd(), data_folder, top_folder)

                            if top_folder == '':
                                print('Please type the directory name!')
                                continue

                            elif os.path.isdir(top_path):
                                print('Gotcha!')
                                print('Your well logging file directory is: %s.' %las_path)
                                print('Your deviation file directory is: %s.' %dev_path)
                                print('Your formation top file directory is: %s.' %top_path)

                                while True:
                                    confirm = input('Are these correct? [Yes/No]: ')

                                    if confirm.lower() == 'yes':
                                        break
                                    
                                    elif confirm.lower() == 'no':
                                        break

                                    else:
                                        print('Please confirm again!')
                                break

                            else:
                                print('Please try again, your directory \'%s\' is not found!' %top_folder)
                        break

                    else:
                        print('Please try again, your directory \'%s\' is not found!' %dev_folder)
                break

            else:
                print('Please try again, your directory \'%s\' is not found!' %las_folder)

    else:
        print('Please try again, your directory \'%s\' is not found!' %data_folder)

In [ ]:
# Function for pairing the data and eliminating the incomplete

def pairing_files(las_files_paths, dev_files_paths, top_files_paths):
    """
    This function is going to pairing the data (las files, dev files and top files) and disable the incompleted one.
    las_files_paths = list of las files with paths
    dev_files_paths = list of deviation files with paths
    top_files_paths = list of formation top files with paths
    """
    paired_las_files_paths = []
    paired_dev_files_paths = []
    paired_top_files_paths = []

    # pairing lAS file to deviation

    for las in las_files_paths:
        for dev in dev_files_paths:
            for top in top_files_paths:

                las_well_name = os.path.basename(las).split('_', 1)[0].lower()
                dev_well_name = os.path.basename(dev).split('_', 1)[0].lower()
                top_well_name = os.path.basename(top).split('_', 1)[0].lower()

                if las_well_name == dev_well_name == top_well_name:
                    paired_las_files_paths.append(las)
                    paired_dev_files_paths.append(dev)
                    paired_top_files_paths.append(top)

    return paired_las_files_paths, paired_dev_files_paths, paired_top_files_paths

# Generate file path

las_files = glob.glob(os.path.join(las_path, '*.las'))
dev_files = glob.glob(os.path.join(dev_path, '*.csv'))
top_files = glob.glob(os.path.join(top_path, '*.csv'))

# Pairing files lAS files, dev files and top files

las_files, dev_files, top_files = pairing_files(las_files, dev_files, top_files)

In [ ]:
# Import lAS files, dev files and top files

lases = [] # Storing well logging data
df_lases = [] # Storing well logging data in panda data frame
devs = [] # Storing deviation data in panda data frame
tops = []# Storing formation top data in panda data frame

for las_file, dev_file, top_file in zip(las_files, dev_files, top_files):

    # Well logging data

    las = lasio.read(las_file)
    lases.append(las)

    # Well logging data in panda data frame

    df = las.df()
    df = df.rename_axis('MD')
    df_lases.append(df)

    # Deviation data in panda data frame

    dev = pd.read_csv(dev_file)
    devs.append(dev)

    # Fomation top data in panda data frame

    top = pd.read_csv(top_file)
    tops.append(top)

# Set directory to save files

sav_folder = 'LQC files'
sav_path = os.path.join(data_path, sav_folder)

if not os.path.isdir(sav_path):
    os.makedirs(sav_path)

In [ ]:
# Fucnction for random bright color list (color map)

def default_colors(n_color):
    """
    This function can generate the list of color code (hex code) following 25 defaults.
    n_color = a number of color
    """
    defaults = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
                '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', 
                '#808080', '#FF0000', '#00FFFF', '#800000', '#008080', 
                '#FFFF00', '#FFBF00', '#0000FF', '#808000', '#000080',
                '#00FF00', '#FF00FF', '#008000', '#800080', '#C0C0C0']

    if int(n_color) > 25:
        colors = defaults.copy()
        
        while len(colors) != int(n_color):
            scrap_code = [''.join([random.choice('0123456789ABCDEF') for i in range(2)]), '00', 'FF']
            random.shuffle(scrap_code)
            color = '#' + ''.join(scrap_code)
            
            if color not in colors:
                colors.append(color)

    else:
        colors = defaults[0:int(n_color)]

    return colors

# Setup well names with color identity

well_names = {}

for las, color in zip(lases, default_colors(len(lases))):
    well_names[las.well['WELL'].value] = color

print('The number of wells is %d.' %len(well_names))
print('Well names are %s.' %', '.join(well_names))

In [ ]:
# Function for checking the curve completion of well

def check_curves(las, alias, mem_curves):
    """
    This function can check the curve completion of well.
    las = las file read by lasio
    alias = curve alias or alterative name of the curve
    mem_curves = necessary curve names for modeling
    """
    curves = [curve.mnemonic for curve in las.curves]
    extracted = []

    for curve in curves:
        for key, values in alias.items():
            if (curve.lower() in [value.lower() for value in values]) & (key in mem_curves):
                extracted.append(key)

    if set(extracted) == set(mem_curves):
        print('All necessary curves in well %s are completed' %las.well['WELL'].value)

    else:
        print('All necessary curves in well %s are incompleted.' %las.well['WELL'].value)
        
        if len(set(extracted).difference(set(mem_curves))) == 1:
            print('Curve %s is missing.' %', '.join([curve for curve in set(mem_curves) - set(extracted)]))

        else:
            print('Curves %s are missing.' %', '.join([curve for curve in set(mem_curves) - set(extracted)]))

# Define standard curve alias for well process

alias = {
'BS' : ['BS', 'BIT'],
'CAL' : ['CAL', 'CALI', 'CALS', 'CLDC'],
'GR' : ['GR', 'GRGC', 'GAM'],
'RHOB' : ['RHOB', 'DEN', 'DENS'],
'NPHI' : ['NPHI', 'NPOR'],
'MSFL' : ['MSFL', 'R20T', 'RSHAL', 'RESS'],
'ILM' : ['ILM', 'R30T', 'R40T', 'R60T', 'RESM'],
'RT' : ['RT', 'R85T', 'LLD', 'RESD'],
'DTC' : ['DTC', 'DT35', 'DT'],
'DTS' : ['DTS', 'DTSM', 'DTSRM', 'DTSXX_COL', 'DTSYY_COL'],
'PEF' : ['PEF', 'PE', 'Pe', 'PDPE']
}

# Define curve name for modeling

mem_curves = ['CAL', 'BS', 'GR', 'RHOB', 'NPHI', 'RT', 'MSFL', 'DTC', 'DTS']

# Define based curve names

based_curves = ['MD', 'AZIMUTH', 'ANGLE', 'BHF']

# Define non affected curves and affected curves for synthetic stage

non_affected = ['RT', 'MSFL', 'GR_NORM']
affected = ['NPHI', 'RHOB', 'DTC', 'DTS'] # element index refers to synthetic ordering

# Check available curves

print('Available curves for each well;')

for las, name in zip(lases, well_names):
    print(name, 'curves are: \n%s' %', '.join([curve.mnemonic for curve in las.curves]))
    check_curves(las, alias, mem_curves)

In [ ]:
# Function for ordering formations from all well data

def merge_sequences(seq1,seq2):
    sm = SequenceMatcher(a = seq1, b = seq2)
    res = []
    
    for (op, start1, end1, start2, end2) in sm.get_opcodes():
        if op == 'equal' or op == 'delete':
            
            #This range appears in both sequences, or only in the first one.
            
            res += seq1[start1:end1]
        elif op == 'insert':
            
            #This range appears in only the second sequence.
            
            res += seq2[start2:end2]
        elif op == 'replace':
            
            #There are different ranges in each sequence - add both.
            
            res += seq1[start1:end1]
            res += seq2[start2:end2]
    
    return res

# Apply function to ordering all formations

forms = []

for top in tops:
    if forms == []:
        for form in top.dropna().Formations:
            forms.append(form)
    else:
        forms = merge_sequences(forms, list(top.dropna().Formations))

# Setup all formations with color identity

all_forms = {}

for form, color in zip(forms, default_colors(len(forms))):
    all_forms[form] = color

In [ ]:
# Function for checking formation names of the input

def check_forms(ref_forms, input_forms):
    """
    This function will check the available formation following the reference and prepare the input for the next step.
    ref_forms = available formation will be checked based on this reference.
    input_names = names of the formation 
    """
    form_names = []

    for form in ref_forms:
        if form.lower() in [name.strip().lower() for name in input_forms.split(',')]:
            form_names.append(form)
    
    return form_names

# Function for arranging the formation following the reference one
 
def forms_arr(ref_forms, app_forms):
    """
    This function will arrange the order of the formation in list following the reference.
    ref_forms = formation order will be arranged following this reference.
    app_forms = list of the formations will be applied.
    """
    for form in ref_forms:
        if form.lower() in [form.lower() for form in app_forms]:
            app_forms.pop([form.lower() for form in app_forms].index(form.lower()))
            app_forms.append(form)
    
    return app_forms

In [ ]:
# Function for adding the formation to the selected formation list

def add_forms(selected_forms, non_selected_forms):
    """
    This function can add more formation to selected formation list.
    selected_forms = the list of selected formations that will be added more by user.
    non_selected_forms = the list of non-selected formations
    *check_forms function is required.
    """
    print('Which one do you want to add more?')

    while True:
        select = input('[Comma can be used for multi-input]: ').strip()
        selected_form = check_forms(non_selected_forms, select)

        if select == '':
            print('Please type formation names!')
            continue

        elif selected_form == []:
            print('Please try again!, formation \'%s\' is not found.' %select)
            continue

        elif set([form.lower() for form in selected_form]).issubset(set([form.lower() for form in non_selected_forms])):
            for form in selected_form:
                selected_forms.append(form)
                non_selected_forms.pop([form.lower() for form in non_selected_forms].index(form.lower()))
            break             
                 
    return selected_forms, non_selected_forms

# Function for removing the formation in the selected formation list

def remove_forms(selected_forms, non_selected_forms):
    """
    This function can remove the seleted formation.
    selected_forms = the list of selected formations that will be removed by user.
    non_selected_forms = the list of non-selected formations
    *check_forms function is required.
    """
    print('Which one do you want to remove?')

    while True:
        delete = input('[Comma can be used for multi-input]: ').strip()
        del_forms = check_forms(selected_forms, delete)

        if delete == '':
            print('Please type formation names!')
            continue

        elif del_forms == []:
            print('Please try again!, formation \'%s\' is not found.' %delete)
            continue       

        elif set([form.lower() for form in del_forms]).issubset(set([form.lower() for form in selected_forms])):
            for form in del_forms:
                selected_forms.pop([form.lower() for form in selected_forms].index(form.lower()))
                non_selected_forms.append(form)

            if len(del_forms) == 1:
                print('Formation %s is removed' %''.join(del_forms))
            
            else:
                print('Formation %s are removed' %', '.join(del_forms))
            break
    
    return selected_forms, non_selected_forms

In [ ]:
# Show all selectable formations

print('All formations in this field are: %s.' %', '.join(all_forms))

# Define selected formations in this project to focus

selected = []
non_selected = forms.copy() # Be used only in this step

while True:
    print('Which one is your selected formation?')
    select = input('[Comma can be used for multi-input]: ').strip()
    selected_form = check_forms(all_forms, select)

    if select == '':
        print('Please type formation names!')
        continue

    elif selected_form == []:
        print('Please try again!, formation \'%s\' is not found.' %select)
        continue

    elif set([form.lower() for form in selected_form]).issubset(set([form.lower() for form in all_forms])):
        
        for form in selected_form:
            selected.append(form)
            non_selected.pop([form.lower() for form in non_selected].index(form.lower()))

        while True:
            selected = forms_arr(all_forms, selected)
            non_selected = forms_arr(all_forms, non_selected)        

            print('Now, only formation \'%s\' will be your selected formations' %', '.join(selected))
            confirm = input('Are you okay with this? [Ok/Not]: ').strip()
            
            if confirm.lower() == 'ok':
                print('Got it, sir/ma\'am!')
                break
            
            elif confirm.lower() == 'not':
                
                while True:
                    options = input('What do you want to do? add more or edit (remove)? [Add/Remove]: ').strip()

                    if options.lower() == 'add':
                        print('The other formation in this field are: %s.' %', '.join(non_selected))
                        selected, non_selected = add_forms(selected, non_selected)
                        break

                    elif options.lower() == 'remove':
                        selected, non_selected = remove_forms(selected, non_selected)

                        if selected == []:
                            print('No formation is selected!, please select formation.')
                            print('The available formation in this field are: %s.' %', '.join(all_forms))
                            selected, non_selected = add_forms(selected, non_selected)
                        break

                    else:
                        print('Please confirm again!')
                        continue
                continue

            else:
                print('Please confirm again!')
                continue
        break

# setup selected formations with color identity

selected_forms = {}

for form in selected:
    if form in all_forms:
        selected_forms[form] = all_forms[form]

In [ ]:
# Define field parameters to adjust (remove air gap) the well logging by oil field type in the next step

confirm = 'no'

while confirm == 'no':

    ground_levels = [] # for onshore field only.
    water_levels = [] # for offshore field only.
    RHOmls = [] # mudline density or surface density for density extrapolation
    air_gaps = []

    field_type = input('What is this oil field type [Onshore/Offshore]: ').strip()

    if field_type.lower() == 'onshore':
        for name in well_names:
            print('Please type basic information for well %s' %name)

            kb = float(input('Kelly Bushing depth (KB level to sea level) [m]: ').strip())
            gl = float(input('Ground elevetion (ground level to sea level) [m]: ').strip())
            RHOml = float(input('Mudline density (density at ground level) [g/c3]: ').strip())
            ground_levels.append(gl)
            RHOmls.append(RHOml)
            air_gaps.append(kb - gl)

        print('Please confirm these; Field type = %s' %(field_type))
        for name, gl, RHOml, ag in zip(well_names, ground_levels, RHOmls, air_gaps):
            print('Well %s; Ground level = %f, Mudline density = %f, Air gap = %f' %(name, gl, RHOml, ag))
        
        while True:
            confirm = input('Are these correct? [Yes/No]: ').strip()

            if confirm.lower() == 'yes':
                break
            
            elif confirm.lower() == 'no':
                break

            else:
                print('Please comfirm again.')
    
    elif field_type.lower() == 'offshore':
        for name, df_las, dev in zip(well_names, df_lases, devs):
            print('Please type basic information for well %s' %name)

            kb = float(input('Kelly Bushing depth (KB level to sea level) [m]: ').strip())
            wl = float(input('Water depth (sea level to seafloor level) [m]: ').strip())
            RHOml = float(input('Mudline density (density at sea level) [g/c3]: ').strip())
            water_levels.append(wl)
            RHOmls.append(RHOml)
            air_gaps.append(kb)

        print('Please confirm these; Field type = %s' %(field_type))
        for name, wl, RHOml, ag in zip(well_names, water_levels, RHOmls, air_gaps):
            print('Well %s; Water level = %f, Mudline density = %f, Air gap = %f' %(name, wl, RHOml, ag))
        
        while True:
            confirm = input('Are these correct? [Yes/No]: ').strip()

            if confirm.lower() == 'yes':
                break
            
            elif confirm.lower() == 'no':
                break

            else:
                print('Please comfirm again.')
    
    else:
        print('Please type only \'Onshore\' or \'Offshore\'')
        continue

In [ ]:
air_gaps

In [ ]:
ground_levels

In [ ]:
RHOmls

In [ ]:
df_lases[0]

In [ ]:
devs[0]

In [ ]:
tops[0]

## Depth conversion

In [ ]:
# Create function for TVD computation by minimum curvature method

def tvd_mini_cuv(dev):
    """
    TVD computation function using minimum curvature survey calculation method
    dev = Deviation survey data in pandas data frame which contains:
             1. Measured depth (MD) in column name "MD"
             2. Azimuth direction (AZIMUTH) in column name "AZIMUTH"
             3. Inclination angle (ANGLE) in column name "ANGLE"
    """
    # setup parameters
    
    md = dev.MD
    prev_md = md.shift(periods = 1, fill_value = 0)
    diff_md = md - prev_md
    
    ang = dev.ANGLE
    prev_ang = ang.shift(periods = 1, fill_value = 0)
    diff_ang = ang - prev_ang
    
    azi = dev.AZIMUTH
    prev_azi = azi.shift(periods = 1, fill_value = 0)
    diff_azi = azi - prev_azi
    
    # computation
    
    cos_theta = np.cos(np.radians(diff_ang)) - (np.sin(np.radians(ang)) * np.sin(np.radians(prev_ang)) * (1 - np.cos(np.radians(diff_azi))))
    theta = np.arccos(cos_theta)
    
    rf = ((2 / theta) * np.tan(theta/2)).fillna(0)
    
    dev['TVD'] = np.cumsum((diff_md / 2) * (np.cos(np.radians(ang)) + np.cos(np.radians(prev_ang))) * rf)
    
    return dev

# Calculate TVD for all well deviations in deviation files

devs = list(map(tvd_mini_cuv, devs))

In [ ]:
# Generate function to convert MD to TVD in data with deviation survey data 

def tvd_inter(las, df_las, dev):
    """
    MD to TVD interpolation using linear interpolation method and update las file
    las = las file (.las) of the well data
    df_las = las input in pandas data frame contains depth column in measured depth (MD)
    dev = deviation survey data in pandas data frame contains depth columns in both measured depth (MD) and true vertical depth (TVD)
    """
    # Merge deviation file with well data
    
    df_las = df_las.reset_index()
    df_las = pd.concat([dev[['MD', 'AZIMUTH', 'ANGLE', 'TVD']], df_las]).sort_values(by = ['MD']).reset_index(drop = True)
    
    # Insert true vertical depth using linear interpolation
    
    for col in df_las[['AZIMUTH', 'ANGLE', 'TVD']].columns:
        df_las[col] = df_las[col].interpolate(method = 'linear', limit_area = 'inside')
        
    # Set true vertical depth as file indices
        
    df_las = df_las.dropna(subset = ['TVD']).set_index('TVD')
    df_las = df_las.drop(list(dev['TVD']))
    
    # Update las files

    las.insert_curve(0, 'TVD', df_las.index, unit = 'm', descr = 'True Vertical Depth', value = '')
    las.insert_curve(1, 'MD', df_las.index, unit = 'm', descr = 'Measured Depth', value = '')
    las.insert_curve(2, 'AZIMUTH', df_las.index, unit = 'degree', descr = 'well Deviation in Azimuth', value = '')
    las.insert_curve(3, 'ANGLE', df_las.index, unit = 'degree', descr = 'well Deviation in Angle', value = '')
    del las.curves['DEPTH']

    print('Measured depth (MD) is converted to True vertical depth (TVD) for well %s' %las.well['WELL'].value)
    
    return las, df_las

In [ ]:
# Generate function to convert MD to TVD in data with deviation survey data 

def tvd_inter_tops(top, dev):
    """
    Formation top and bottom depth in MD to TVD interpolation using linear interpolation method and update top file
    top = formation top file contains Formation name (Formations), Top depth (Top) and bottom depth (Bottom).
    dev = deviation survey data in pandas data frame contains depth columns in both measured depth (MD) and true vertical depth (TVD)
    """
    columns = {'Top':'Top_TVD', 'Bottom':'Bottom_TVD'}

    for col in columns:
        top['MD'] = top[col]
        top = pd.concat([dev[['MD', 'TVD']], top]).sort_values(by = ['MD']).reset_index(drop = True)
        top['TVD'] = top['TVD'].interpolate(method = 'linear', limit_area = 'inside')
        top = top.dropna(subset = ['Formations']).reset_index(drop = True)
        top = top.drop(columns = ['MD', col]).rename(columns = {'TVD':columns[col]})

        cols = top.columns.tolist()
        cols = cols[1:] + cols[:1]

        top = top[cols]

    top['Bottom_TVD'] = top['Bottom_TVD'].fillna(dev['TVD'].max())
        
    return top

In [ ]:
# Function for adjust true vertical depth of well logging data

def remove_gap(tvd_las, dev, tvd_top, gap):
    """
    This function can remove air gap from the true vertical depth (TVD) for well logging data.
    tvd_las = las input in pandas data frame contains depth column in true vertical depth (TVD)
    gap = air gap value (onshore = kelly bushing - ground level, offshore = kelly bushing)
    """
    tvd_las = tvd_las.reset_index()
    tvd_las['TVD'] = tvd_las['TVD'] - gap
    tvd_las['MD'] = tvd_las['MD'] - gap
    tvd_las = tvd_las.set_index('TVD')

    dev['TVD'] = dev['TVD'] - gap
    dev['MD'] = dev['MD'] - gap

    tvd_top['Top_TVD'] = tvd_top['Top_TVD'] - gap
    tvd_top['Bottom_TVD'] = tvd_top['Bottom_TVD'] - gap

    return tvd_las, dev, top

# Implement interpolation function

tvd_lases = []
tvd_tops = []

for las, df_las, dev, top, gap in zip(lases, df_lases, devs, tops, air_gaps):
    las, df_las = tvd_inter(las, df_las, dev)
    top = tvd_inter_tops(top.copy(), dev)
    
    tvd_las, dev, tvd_top = remove_gap(df_las, dev, top, gap)
    
    tvd_lases.append(tvd_las)
    tvd_tops.append(tvd_top)

## Bad Hole Flag (BHF)

In [ ]:
# Function for create Bad Hole flag

def create_bhf(las, tvd_las, alias):
    """
    This function can compute Bad Hole Flag using confidential interval (ci) and update las file
    las = las file (.las) of the well data
    tvd_las = well logging data in data frame (converted true vertical depth)
    * Caliper and bitsize data are required.
    """
    # ci = confidential interval factor (0.00-1.00, default = 0.75)
    
    ci = 0.75 # default

    for col in tvd_las.columns:
        if col in alias['CAL']:
            caliper = col
        elif col in alias['BS']:
            bitsize = col

    confirm = 'not'

    while not confirm.lower() == 'ok':
        diff = tvd_las[caliper] - tvd_las[bitsize]
        interval = st.norm.interval(alpha = ci, loc = round(np.mean(diff), 2), scale = round(np.std(diff), 2))
        print(interval[0])
        print(interval[1])

        # tvd_las['BHF'] = (diff.dropna() > interval[0]) & (diff.dropna() < interval[1])
        tvd_las['BHF'] = (diff.dropna() > -0.15) & (diff.dropna() < 0.15)
        tvd_las['BHF'] = tvd_las['BHF']*1
        tvd_las['BHF'] ^= 1

        #create figure

        fig, axis = plt.subplots(nrows = 1, ncols = 2, figsize = (6, 10))
        fig.suptitle(las.well['WELL'].value, fontsize = 15, y = 1.0)

        #General setting for all axis
        
        for ax in axis:
            ax.set_ylim(tvd_las.index.min(), tvd_las.index.max())
            ax.invert_yaxis()
            ax.minorticks_on() #Scale axis
            ax.get_xaxis().set_visible(False)
            ax.grid(which = 'major', linestyle = '-', linewidth = '0.5', color = 'green')
            ax.grid(which = 'minor', linestyle = ':', linewidth = '0.5', color = 'black') 

        # caliper - bitsize plot
        
        ax11 = axis[0].twiny()
        ax11.set_xlim(6,15)
        ax11.plot(tvd_las[bitsize], tvd_las.index, color = 'black')
        ax11.spines['top'].set_position(('outward',0))
        ax11.set_xlabel('BS[in]',color = 'black')
        ax11.tick_params(axis = 'x', colors = 'black')
        
        ax12 = axis[0].twiny()
        ax12.set_xlim(6,15)
        ax12.plot(tvd_las[caliper], tvd_las.index, color = 'grey' )
        ax12.spines['top'].set_position(('outward',40))
        ax12.set_xlabel('CAL[in]',color = 'grey')
        ax12.tick_params(axis = 'x', colors = 'grey')

        ax12.grid(True)

        # Bad Hole Flag plot
    
        ax21 = axis[1].twiny()
        ax21.plot(tvd_las['BHF'], tvd_las.index, color = 'red')
        ax21.fill_betweenx(tvd_las.index, 0, tvd_las['BHF'], color = 'red', label = 'Bad hole')
        ax21.spines['top'].set_position(('outward',0))
        ax21.set_xlabel('BHF', color = 'red')
        ax21.tick_params(axis = 'x', colors = 'red')
        ax21.set_xticks([0, 1])
        ax21.set_xticklabels(['GOOD', 'BAD'])
        ax21.legend(loc = 'upper right')
        
        ax21.grid(True)
        
        fig.tight_layout()
        
        plt.show()

        while True:
            confirm = input('Are you ok with this created bad hole flag? [Ok/Not]: ').strip()   

            if confirm.lower() == 'ok':
                # las.append_curve('BHF', tvd_las['BHF'], unit = 'unitless', descr = 'Bad Hole Flag', value = '')
                break

            elif confirm.lower() == 'not':
                
                while True:
                    ci = float(input('Please type value between 0.00 - 1.00 (0.75 is default) to adjust: ').strip())
                
                    if 0 < ci < 1:
                        break

                    else:
                        print('Input value is out of range!')
                break

            else:
                print('please confirm again!')

    return las, tvd_las

# Create Bad Hole flag for each well

for las, tvd_las in zip(lases, tvd_lases):
    las, tvd_las = create_bhf(las, tvd_las, alias)

## Quality Control 1

In [ ]:
# Function for initial plot for first inspection

def initial_inspection(las, tvd_las, inspect_name):
    """
    For all curve initial inspection
    las = las file (.las) of the well data
    tvd_las = well logging data in data frame (converted true vertical depth)
    inspect_name = name of saved figure
    """
    # Create figure

    fig, axis = plt.subplots(nrows = 1, ncols = len(tvd_las.columns), figsize = (30,20), sharey = True)
    fig.suptitle(las.well['WELL'].value, fontsize = 30, y = 1.0)
    
    units = [curve.unit for curve in las.curves]
    units.pop(0)

    # Plot setting for all axis

    for ax, col, unit in zip(axis, tvd_las.columns, units):
        ax.set_ylim(tvd_las.index.min(), tvd_las.index.max())
        ax.invert_yaxis()
        ax.minorticks_on() #Scale axis
        ax.grid(which = 'major', linestyle = '-', linewidth = '0.5', color = 'green')
        ax.grid(which = 'minor', linestyle = ':', linewidth = '0.5', color = 'black')
        ax.set_xlabel(col + '\n(%s)' %unit, fontsize = 15)

        if (col in alias['RT']) or (col in alias['ILM']) or (col in alias['MSFL']):
            ax.plot(tvd_las[col], tvd_las.index)
            ax.set_xscale('log')

        elif col == 'BHF':
            ax.plot(tvd_las[col], tvd_las.index)
            ax.fill_betweenx(tvd_las.index, 0, tvd_las[col], label = 'Bad hole')
        
        else:
            ax.plot(tvd_las[col], tvd_las.index)

    fig.tight_layout()

    # Save files

    inspect_folder = 'LQC_Inspect'
    inspect_path = os.path.join(sav_path, inspect_folder)

    if not os.path.isdir(inspect_path):
        os.makedirs(inspect_path)

    plt.savefig(os.path.join(inspect_path, inspect_name), dpi = 200, format = 'png', bbox_inches = "tight")

    plt.show()

# Plot available curves

for las, tvd_las in zip(lases, tvd_lases):
    inspect_name = 'LQC_' + las.well['WELL'].value + '_Inspect' + '.png'
    initial_inspection(las, tvd_las, inspect_name)

In [ ]:
# Function for removing some zone.

def eliminate_zone(tvd_las, alias, top_zone, bottom_zone, based_curves):
    """
    This function is for data elimination within the interested zone. 
    All curves within that zone will be removed.
    Except measured depth (MD), azimuth direction (AZIMUTH), angle or deviation (ANGLE), bitsize (BS), caliper (CAL) and bad hole flag (BHF).
    tvd_las = well logging data in pandas data frame in TVD depth.
    alias = curve alias or alterative name of the curve.
    top_zone = Top TVD depth of the zone you want to remove.
    bottom_zone = Bottom TVD depth of the zone you want to remove.
    based_curves = list of based curve names
    """
     # Set data columns for elimination

    data_cols = tvd_las.columns
    base_cols = based_curves.copy()
    edit_cols = []

    for bs in alias['BS']:
        base_cols.append(bs)

    for cal in alias['CAL']:
        base_cols.append(cal)

    for col in data_cols:
        if col not in base_cols:
            edit_cols.append(col)

    # Eliminate the data within the assigned interval

    tvd_las.loc[(tvd_las.index > float(top_zone)) & (tvd_las.index < float(bottom_zone)), edit_cols] = np.nan

    return tvd_las

In [ ]:
# Define starting point of the accepted data

starting_points = []

for las, tvd_las in zip(lases, tvd_lases):
    depth_min = tvd_las.index.min() 
    depth_max = tvd_las.index.max()
    print('For well %s, it has logging data between depth %.2f to %.2f.' %(las.well['WELL'].value, depth_min, depth_max))

    while True:
        starting_point = float(input('Please indicate a starting point of the accepted logging data depth.').strip())

        if (starting_point >= depth_min) & (starting_point < depth_max):
            tvd_las = eliminate_zone(tvd_las, alias, (depth_min - las.well['STEP'].value), starting_point, based_curves)
            inspect_edited_name = 'LQC_' + las.well['WELL'].value + '_Inspect_Edited.png'
            initial_inspection(las, tvd_las, inspect_edited_name)
            starting_points.append(starting_point)
            break

        else:
            print('Your input is out of range')
            continue

In [ ]:
# Define well and zone of messy data

eliminated_zones = []

while True:
    print('Are there any depth interval you would like to remove or delete?')
    answer = input('Please type Yes or No [Yes/No]: ').strip()

    if answer.lower() == 'yes':

        while True:
            print('There are %d wells.' %len(well_names))
            print('%s, Which one you want to edit?' %', '.join(well_names))
            name = input('Please indicate the well name you want to edit: ').strip()         
            
            if name.lower() in [name.lower() for name in well_names]:
                i = [name.lower() for name in well_names].index(name.lower())

                while True:
                    depth_min = tvd_lases[i].index.min()
                    depth_max = tvd_lases[i].index.max()
                    print('This well has logging data between depth %.2f to %.2f.' %(depth_min, depth_max))
                    top_zone = float(input('Please indicate top depth of the zone or interval you want to edit in TVD depth: ').strip())

                    if (top_zone >= depth_min) & (top_zone < depth_max):

                        while True:
                            bottom_zone = float(input('Please indicate bottom depth of the zone or interval you want to edit in TVD depth: ').strip())
                            
                            if (bottom_zone > top_zone) & (bottom_zone <= depth_max):
                                
                                while True:
                                    print('The data of well', lases[i].well['WELL'].value, 'in TVD depth from', top_zone, 'to', bottom_zone, 'will be eliminated.')
                                    confirm = input('Are you sure? [Yes/No]: ').strip() 
                                    
                                    if confirm.lower() == 'yes':
                                        tvd_lases[i] = eliminate_zone(tvd_lases[i], alias, top_zone, bottom_zone, based_curves)
                                        inspect_edited_name = 'LQC_' + lases[i].well['WELL'].value + '_Inspect_Edited.png'
                                        initial_inspection(lases[i], tvd_lases[i], inspect_edited_name)

                                        zone = {}
                                        zone[i] = '%f, %f' %(top_zone, bottom_zone)
                                        eliminated_zones.append(zone)

                                        print('The data has been eliminated.')
                                        break
                                        
                                    elif confirm.lower() == 'no':
                                        break

                                    else:
                                        print('Please comfirm again!')
                                        continue
                                break
                                        
                            else:
                                print('Your bottom depth is out of range')
                                continue
                        break

                    else:
                        print('Your top depth is out of range')
                        continue
                break

            else:
                print('Your well %s is not found., Please select again!' %name)
                continue
        continue

    elif answer.lower() == 'no':
        print('Noted, Sir/Ma\'am!')
        break

    else:
        print('Please comfirm again!')
        continue

In [ ]:
# Function for standardizing or renaming based on alias

def apply_alias(las, tvd_las, alias):
    """
    This function is going to rename curves based on alias. The duplicates will be named followed by the number.
    las = las file (.las) of the well data
    tvd_las = well logging data in pandas data frame in TVD depth.
    alias = curve alias or alterative name of the curve.
    """
    # Get standard curve name from alias

    new_cols = {}
    seen = {}
    dupes = []

    for col in tvd_las.columns:
        for key, values in alias.items():
            
            if col in values:
                new_col = key

                if key not in seen:
                    seen[key] = 1
                
                else:
                    if seen[key] == 1:
                        dupes.append(key)

                    seen[key] += 1
                    new_col = "{}_{}".format(key, seen[key])

                new_cols[col] = new_col

    # Apply to tvd_las

    tvd_las = tvd_las.rename(columns = new_cols)

    # Apply to las

    for key, value in new_cols.items():
        las.curves[key].mnemonic = value

    print('All curve names of well %s are standardized' %las.well['WELL'].value)
    
    return las, tvd_las, seen, dupes

In [ ]:
# Function for setting up the well logging data without the duplicate

def setup_curve(las, well, seen, dupes, mem_curves, based_curves):
    """
    This function will select and eliminate curve data for setting up modeling curve inputs.
    las = las file (.las) of the well data
    well = well logging data in pandas data frame in TVD depth with alias applied.
    seen = dictionary contains the name and number of curve
    dupes = list of duplicated curve name
    mem_curves = necessary curve names for modeling
    based_curves = list of based curve names
    """
    # Select modeling curves

    for col in well.columns:
        if col.split('_')[0] not in (based_curves + mem_curves):
            well = well.drop([col], axis=1)
            del las.curves[col]

    # Manage duplicate curves

    new_col = {}
    choices = []

    for key, value in seen.items():
        if (key in mem_curves) & (key in dupes):
            print('%d curves of %s are found as duplicated curves for well %s' %(value, key, las.well['WELL'].value))

            for num in range(value):
                if num == 0:
                    curve = las.curves[key]
                    print('%s curve is %s' %(curve.mnemonic, curve.descr))
                    choices.append(key)
                else:
                    curve = las.curves[key + '_' + str(num+1)]
                    print('%s curve is %s' %(curve.mnemonic, curve.descr))
                    choices.append(key + '_' + str(num+1))

            while True:
                select = input('Please select a representative curve for %s: ' %key).strip()

                if select.lower() in [choice.lower() for choice in choices]:
                    index = [choice.lower() for choice in choices].index(select.lower())
                    new_col[choices[index]] = key
                    choices.pop(index)
                    break

                else:
                    print('Please type again!, your curve %s is not found.' %select)
                    continue
            
    # Eliminate duplicates

    for col in choices:
        well = well.drop([col], axis=1)
        del las.curves[col]
            
    # Set curve name
    
    well = well.rename(columns = new_col)

    for key, value in new_col.items():
        las.curves[key].mnemonic = value

    print('All curve data of well %s are setup already' %las.well['WELL'].value)

    return las, well

# Rename curve and setup curve for modeling

wells = []

print('The system is standardizing and setting up the curves.')

for las, tvd_las in zip(lases, tvd_lases):
    las, well, seen, dupes = apply_alias(las, tvd_las.copy(), alias)
    las, well = setup_curve(las, well, seen, dupes, mem_curves, based_curves)
    wells.append(well)

In [ ]:
# Function for controling the data by confidential interval (ci)

def data_control(well):
    """
    This function can limit exceeded value of the data by using confidential interval (ci)
    well = well logging data in pandas data frame in TVD depth with alias applied.
    """
    # ci = confidential interval factor (0.00-1.00, default = 0.95)

    ci = 0.95

    not_apply_cols = based_curves + ['BS', 'CAL']

    for col in well.columns:
        if col not in not_apply_cols:
            interval = st.norm.interval(alpha = ci, loc = round(np.mean(well[col]), 2), scale = round(np.std(well[col]), 2))
            well.loc[(well[col] < interval[0]) | (well[col] > interval[1]), col] = np.nan
    
    return well

# Function for eliminating bad data using bad hole flag

def bhf_control(well, affected):
    """
    This function can eliminate the affected data within the bad zone including density (RHOB), neutron porosity (NPHI), P-sonic (DTC) and S-sonic (DTS).
    well = well logging data in pandas data frame in TVD depth with alias applied.
    affected = list of affected curve names.
    *Bad hole flag must be created using create_bhf function
    """
    # Eliminate the data based on bad hole flag

    for col in affected:
        well.loc[((well[col] * (well.BHF ^ 1)) == 0), [col]] = np.nan

    return well

# Apply the functions to limit and eliminate the data to be ready for synthetic stage

for well in wells:
    well = data_control(well)
    well = bhf_control(well, affected)

## Data synthetic

In [ ]:
# Normalize gamma ray log for data synthetic

def norm_gr(las, well):
    """
    This function is used for min-max normalization calculation for well synthetic.
    las = las file (.las) of the well data
    well = well logging data in pandas data frame in TVD depth with alias applied.
    """
    # Normolize gamma ray curve

    well['GR_NORM'] = (well.GR - np.min(well.GR)) / (np.max(well.GR) - np.min(well.GR))

    # Update las file

    las.append_curve('GR_NORM', well.GR_NORM, unit = 'unitless', descr = 'Normalized Gamma Ray', value = '')

    return las, well

# Apply normalization

for las, well in zip(lases, wells):
    las, well = norm_gr(las, well)

In [ ]:
# Function for Checking normalized gamma ray

def hist_norm_gr(wells, well_names, norm_name):
    """
    This function can plot histogram of normalized gamma ray data from all well.
    wells = list of well logging data in pandas data frame in TVD depth with alias applied.
    well_names = list of well name in this field
    norm_name = name of saved figure
    *Normalized Gamma Ray must be calculated using norm_gr function.
    """
    # bins = a number of histogram bar (default = 150)

    bins = 150

    # Create figure
    
    fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20,10), sharey = True)
    fig.suptitle('Histogram of Normalized Gamma Ray', fontsize= 15, y = 0.98)
    
    # Plot histrogram
    for well, name in zip(wells, well_names):
        ax[0].hist(well.GR, bins = bins, histtype = 'step', label = name, color = well_names[name])
        ax[0].set_xlabel('Gamma Ray (API)')
        ax[0].set_ylabel('Frequency')
        ax[0].set_title('Before')
        ax[0].legend(loc='upper left')
        
        ax[1].hist(well.GR_NORM, bins = bins, histtype = 'step', label = name, color = well_names[name])
        ax[1].set_xlabel('Normalized Gamma Ray')
        ax[1].set_ylabel('Frequency')
        ax[1].set_title('After')
        ax[1].set_xlim([0,1])
        ax[1].legend(loc='upper left')

    fig.tight_layout()

    # Save files

    synthetic_folder = 'LQC_Synthetic'
    synthetic_path = os.path.join(sav_path, synthetic_folder)

    if not os.path.isdir(synthetic_path):
        os.makedirs(synthetic_path)

    plt.savefig(os.path.join(synthetic_path, norm_name), dpi = 200, format = 'png', bbox_inches = "tight")

    plt.show()
        
# Check normalized gamma ray

norm_name = 'LQC_Norm_GR.png'
hist_norm_gr(wells, well_names, norm_name)

In [ ]:
# Function for creating the data set of synthetic modeling

def set_data(wells, non_affected, affected):
    """
    This function can create the data set for model training and testing of synthetic function.
    wells = list of well logging data in pandas data frame in TVD depth with alias applied.
    non_affected = list of non affected curve names for synthetic.
    affected =  list of affected curve names for synthetic.
    """
    # Indicate curves for data set

    req_curves = non_affected.copy() + affected.copy()

    # Build an empty data set and collect the data from each well
    
    data_set = pd.DataFrame()
    
    for well in wells:
        data_set = pd.concat([data_set, well[req_curves]])
            
    data_set = data_set.dropna()
            
    return data_set

# Generate the data set for synthetic stage

data_set = set_data(wells, non_affected, affected)

In [ ]:
# Function for curve synthetic (filled nan with mean)

def well_syn(las, well, data_set, non_affected, affected):
    """
    This function can synthesize bad data within bad zone indicated by bad hole flag.
    This function is going to fix or synthetic the curve one at the time until all curve are fixed.
    This function based on machine learning techniques (default = multilinear regression and random forest regression)
    
    RT, MSFL and GR_NORM are used as initial curves.
    NPHI, RHOB, DTC and DTS will be synthesized respectively.

    Neutron porosity synthesizing using;
    1.) Deep resistivity (RT)
    2.) Shallow resistivity (MSFL)
    3.) Normalized gamma ray (GR_NORM)
    
    Density synthesizing using;
    1.) Deep resistivity (RT)
    2.) Shallow resistivity (MSFL)
    3.) Normalized gamma ray (GR_NORM)
    4.) Neutron porosity (NPHI)
    
    P-Sonic synthesizing using;
    1.) Deep resistivity (RT)
    2.) Shallow resistivity (MSFL)
    3.) Normalized gamma ray (GR_NORM)
    4.) Neutron porosity (NPHI)
    5.) Density (RHOB)
    
    S-Sonic synthesizing using;
    1.) Deep resistivity (RT)
    2.) Shallow resistivity (MSFL)
    3.) Normalized gamma ray (GR_NORM)
    4.) Neutron porosity (NPHI)
    5.) Density (RHOB)
    6.) P-Sonic (DTC)
    
    las = las file (.las) of the well data
    well = well logging data in pandas data frame in TVD depth with alias applied.
    data_set = data set for model training and testing in pandas data frame.
    *data_set can be created using set_data function.
    non_affected = list of non affected curve names for synthetic.
    affected =  list of affected curve names for synthetic. 
    *The element index of affected curve names will affect synthetic ordering.
    """
    # test_size = size of test data for modeling (0.00 - 1.00, default = 0.3)
    
    test_size = 0.3

    # n_tree = number of decision tree in random forest regression technique (default = 10)

    n_tree = 10

    # Set initial and synthesized data

    initial = non_affected.copy()
    syns = affected.copy()

    cols = initial.copy()

    print('System is synthesizing the data for well %s' %las.well['WELL'].value)

    # Synthesize data one at the time

    for syn in syns:

        r2 = {}
        
        # Split the data
        
        input_training = data_set[initial]
        output_traning = data_set[syn]
        
        X_train, X_test, y_train, y_test = train_test_split(input_training, output_traning, test_size = test_size, random_state = 0)

        # Setup synthesizing input (Nan values are filled with mean column values)

        syn_input = well[cols].fillna(well[cols].mean())
        
        # Multilinear regression modeling

        mlr = LinearRegression()
        mlr.fit(X_train, y_train)

        mlr_r = mlr.score(X_test, y_test)
        r2[mlr_r] = [mlr.predict(syn_input), 'Multilinear Regression', mlr_r]

        # Random forest regression modeling

        rfr = RandomForestRegressor(n_estimators = n_tree)
        rfr.fit(X_train, y_train)

        rfr_r = rfr.score(X_test, y_test)
        r2[rfr_r] = [rfr.predict(syn_input), 'Random Forest Regression', rfr_r]

        # Select the best regression

        syn_output = best_r2(r2)[0]
        print('%s is implemented for %s with R-squared value %f' %(best_r2(r2)[1], syn, best_r2(r2)[2]))

        well[syn + '_SYN'] = pd.DataFrame(syn_output, index = well[cols].index)
        
        # Merge curve where synthetic curve replace bad hole sections, and good original curve data remains in place
        
        well[syn + '_MRG'] = well[syn].fillna(well[syn + '_SYN'], inplace = False)
                
        # Iterate new syntheric curve with new initial curves
        
        initial.append(syn)
        cols.append(syn + '_SYN')

    # Update las file

    las.append_curve('NPHI_SYN', well['NPHI_SYN'], unit = las.curves['NPHI'].unit, descr = 'Synthetic neutron porosity', value = '')
    las.append_curve('NPHI_MRG', well['NPHI_MRG'], unit = las.curves['NPHI'].unit, descr = 'Merged neutron porosity', value = '')

    las.append_curve('RHOB_SYN', well['RHOB_SYN'], unit = las.curves['RHOB'].unit, descr = 'Synthetic density', value = '')
    las.append_curve('RHOB_MRG', well['RHOB_MRG'], unit = las.curves['RHOB'].unit, descr = 'Merged density', value = '')

    las.append_curve('DTC_SYN', well['DTC_SYN'], unit = las.curves['DTC'].unit, descr = 'Synthetic P-sonic', value = '')
    las.append_curve('DTC_MRG', well['DTC_MRG'], unit = las.curves['DTC'].unit, descr = 'Merged P-sonic', value = '')

    las.append_curve('DTS_SYN', well['DTS_SYN'], unit = las.curves['DTS'].unit, descr = 'Synthetic S-sonic', value = '')
    las.append_curve('DTS_MRG', well['DTS_MRG'], unit = las.curves['DTS'].unit, descr = 'Merged S-sonic', value = '')
    
    return las, well

In [ ]:
# Function for selecting the best value

def best_r2(r2):
    """
    This function can select the best element from dictionary by the highest r-squared value.
    r2 = r-squared value with the elements in dictionary form.
    """
    max = list(r2.keys())[0]

    for x in r2: 
        if x > max : 
             max = x 
      
    return r2[max]

# Synthesize the data

for las, well, point in zip(lases, wells, starting_points):
    las, well = well_syn(las, well, data_set, non_affected, affected)
    well = eliminate_zone(well, alias, (well.index.min() - las.well['STEP'].value), point, based_curves)

for zone in eliminated_zones:
    for key, value in zone.items():
        top = zone[key].split(', ')[0]
        bottom = zone[key].split(', ')[1]
        wells[key] = eliminate_zone(wells[key], alias, top, bottom, based_curves)

print('Data synthesizing is done.')

In [ ]:
# Function for ploting comparision between before and after synthesizing

def syn_compare(las, well, syn_name):
    """
    This function shows ploting between before and after synthesizing.
    las = las file (.las) of the well data
    well = well logging data in pandas data frame in TVD depth with alias applied.
    syn_name = name of saved figure
    """
    # Create figure
    
    fig, axis = plt.subplots(nrows = 1, ncols = 5, figsize = (12,20), sharey = True)
    fig.suptitle(las.well['WELL'].value, fontsize= 20, y = 1)
    
    #General setting for all axis
    
    for ax in axis:
        ax.set_ylim(well.index.min(), well.index.max())
        ax.invert_yaxis()
        ax.minorticks_on() #Scale axis
        ax.grid(which='major', linestyle='-', linewidth='0.5', color='green')
        ax.grid(which='minor', linestyle=':', linewidth='0.5', color='blue') 
    
    # Neutron porosity plot
    
    axis[0].plot(well.NPHI_SYN, well.index, linewidth='0.8', color = 'red', label = 'NPHI_SYN')
    axis[0].plot(well.NPHI, well.index, linewidth='0.8', color = 'blue', label = 'NPHI')
    axis[0].plot(well.NPHI_MRG, well.index, linewidth='0.8', color = 'black', label = 'NPHI_MRG', linestyle = '--')
    axis[0].set_xlim(-0.15, 0.45)
    axis[0].set_xlabel('(%s)\nCorrelation : %.2f' %(las.curves['NPHI'].unit, well.NPHI.corr(well.NPHI_SYN)))    
    axis[0].legend(loc = 'upper left')
    
    # Density plot
    
    axis[1].plot(well.RHOB_SYN, well.index, linewidth='0.8', color = 'red', label = 'RHOB_SYN')
    axis[1].plot(well.RHOB, well.index, linewidth='0.8', color = 'blue', label = 'RHOB')
    axis[1].plot(well.RHOB_MRG, well.index, linewidth='0.8', color = 'black', label = 'RHOB_MRG', linestyle = '--')
    axis[1].set_xlim(1.95, 2.95)
    axis[1].set_xlabel('(%s)\nCorrelation : %.2f' %(las.curves['RHOB'].unit, well.RHOB.corr(well.RHOB_SYN)))    
    axis[1].legend(loc = 'upper left')
    
    # P-sonic plot
    
    axis[2].plot(well.DTC_SYN, well.index, linewidth='0.8', color = 'red', label = 'DTC_SYN')
    axis[2].plot(well.DTC, well.index, linewidth='0.8', color = 'blue', label = 'DTC')
    axis[2].plot(well.DTC_MRG, well.index, linewidth='0.8', color = 'black', label = 'DTC_MRG', linestyle = '--')
    axis[2].set_xlim(40, 140)
    axis[2].set_xlabel('(%s)\nCorrelation : %.2f' %(las.curves['DTC'].unit, well.DTC.corr(well.DTC_SYN)))    
    axis[2].legend(loc = 'upper left')
    
    # S-sonic plot
    
    axis[3].plot(well.DTS_SYN, well.index, linewidth='0.8', color = 'red', label = 'DTS_SYN')
    axis[3].plot(well.DTS, well.index, linewidth='0.8', color = 'blue', label = 'DTS')
    axis[3].plot(well.DTS_MRG, well.index, linewidth='0.5', color = 'black', label = 'DTS_MRG', linestyle = '--')
    axis[3].set_xlim(40, 340)
    axis[3].set_xlabel('(%s)\nCorrelation : %.2f' %(las.curves['DTS'].unit, well.DTS.corr(well.DTS_SYN)))
    axis[3].legend(loc = 'upper left')

    # Bad Hole Flag plot

    axis[4].plot(well['BHF'], well.index, color = 'red')
    axis[4].fill_betweenx(well.index, 0, well['BHF'], color = 'red', label = 'Bad hole')
    axis[4].set_xlabel('BHF')
    axis[4].legend(loc = 'upper left')
        
    fig.tight_layout()

    # Save files

    synthetic_folder = 'LQC_Synthetic'
    synthetic_path = os.path.join(sav_path, synthetic_folder)

    if not os.path.isdir(synthetic_path):
        os.makedirs(synthetic_path)

    plt.savefig(os.path.join(synthetic_path, syn_name), dpi = 200, format = 'png', bbox_inches = "tight")

    plt.show()
        
# Check the synthetic

for las, well in zip(lases, wells):
    syn_name = 'LQC_' + las.well['WELL'].value + '_Synthetic' + '.png'
    syn_compare(las, well, syn_name)

## Quality Control 2 by Boxplot

In [ ]:
# Function for check the quality of the input data for interested zone

def qc_data(wells, tvd_tops, formation, well_names):
    """
    This function will create the boxplot for checking the input data.
    wells = completed well data in pandas dataframe (Merged data with the synthetics)
    tvd_top = formation top data in pandas data frame (MD depth is converted to TVD.) which contains:
            1. Formation names in column name "Formations"
            2. Top depth boundary of the formation in column name "Top_TVD"
            3. Bottom depth boundary of the formation in column name "Bottom_TVD"
    formation = input the name of the formation where the data can be compared
    well_names = list of well names with color code in dictionary format
    """
    # Set data for specific interval
    
    GR_plot = []
    RHOB_plot = []
    NPHI_plot = []
    DTC_plot = []
    DTS_plot = []

    well_labels = []
    
    curve_labels = ['GR', 'RHOB', 'NPHI', 'DTC', 'DTS']
    
    for well, tvd_top, name in zip(wells, tvd_tops, well_names):
        
        # Check available data for selected formation
        
        if formation in list(tvd_top.Formations):
            
            # Set interval from each well for selected formation
            
            top_depth = float(tvd_top.loc[tvd_top.Formations == formation].Top_TVD)
            bottom_depth = float(tvd_top.loc[tvd_top.Formations == formation].Bottom_TVD)

            well_labels.append(name)
            
            # Select data from each well by interval
        
            GR = well.GR.loc[(well.index > top_depth) & (well.index < bottom_depth)].dropna()
            RHOB = well.RHOB_MRG.loc[(well.index > top_depth) & (well.index < bottom_depth)].dropna()
            NPHI = well.NPHI_MRG.loc[(well.index > top_depth) & (well.index < bottom_depth)].dropna()
            DTC = well.DTC_MRG.loc[(well.index > top_depth) & (well.index < bottom_depth)].dropna()
            DTS = well.DTS_MRG.loc[(well.index > top_depth) & (well.index < bottom_depth)].dropna()
        
            GR_plot.append(GR)
            RHOB_plot.append(RHOB)
            NPHI_plot.append(NPHI)
            DTC_plot.append(DTC)
            DTS_plot.append(DTS)

    # Setup well colors for plotting

    well_colors = []

    for name in well_labels:
        well_colors.append(well_names[name])

    well_colo = [item for sublist in [(c, c) for c in well_colors] for item in sublist]
    
    # Create figure
    
    fig, axis = plt.subplots(nrows = 1, ncols = 5, figsize = (22, 5), sharey = False)
    fig.suptitle('Box Plot Quality Control of formation ' + '\'' + formation + '\'', fontsize= 12, y = 1.0)
    
    # Plot setting for all axis
    
    data_plots = [GR_plot, RHOB_plot, NPHI_plot, DTC_plot, DTS_plot]
    
    for data, label, ax in zip(data_plots, curve_labels, axis):
        boxes = ax.boxplot(data, labels = well_labels, meanline = True, notch = True, showfliers = False, patch_artist = True)
        
        # set decoration
        for patch, color in zip(boxes['boxes'], well_colors): 
            patch.set_facecolor(color) 
        
        for box_wk, box_cap, color in zip(boxes['whiskers'], boxes['caps'], well_colo):
            box_wk.set(color = color, linewidth = 1.5)
            box_cap.set(color = color, linewidth = 3)
        
        for median in boxes['medians']:
            median.set(color = 'black', linewidth = 3) 
            
        ax.set_title(label)
    
    fig.tight_layout()

    # Save files

    qc_folder = 'LQC_Boxplot'
    qc_path = os.path.join(sav_path, qc_folder)

    if not os.path.isdir(qc_path):
        os.makedirs(qc_path)

    plt.savefig(os.path.join(qc_path, qc_name), dpi = 200, format = 'png', bbox_inches = "tight")

    plt.show()

# Plot boxplot for quality comtrol each selected formation

for form in selected_forms:
    qc_name = 'LQC_' + form + '.png'
    qc_data(wells, tvd_tops, form, well_names)

## Data visualization

In [ ]:
# Function for well data visualization in composite log plots

def composite_logs(las, well, tvd_top, all_forms, logs_name):
    """
    Plot the curves in composite logs
    las = las file (.las) of the well data
    well = well logging data in pandas data frame in TVD depth with alias applied.
    tvd_top = formation top data in pandas data frame (MD depth is converted to TVD.)
    all_forms = list of all formation names with color code in dictionary format
    logs_name = name of saved figure.
    """
    # Create figure and subplots
    
    fig, axis = plt.subplots(nrows = 1, ncols = 6, figsize = (15,20), sharey = True)
    fig.suptitle(las.well['WELL'].value, fontsize= 20, y = 1.0)
    
    # General setting for all axis
    
    for ax in axis:
        ax.set_ylim(well.index.min(), well.index.max())
        ax.invert_yaxis()
        ax.minorticks_on() #Scale axis
        ax.get_xaxis().set_visible(False) 
        ax.grid(which = 'major', linestyle = '-', linewidth = '0.5', color = 'green')
        ax.grid(which = 'minor', linestyle = ':', linewidth = '0.5', color = 'blue')
        
        # Plot formations
        
        for top, form in zip(tvd_top.Top_TVD, tvd_top.Formations):
            if ((top >= well.index.min()) and (top <= well.index.max())):
                ax.axhline(y = top, linewidth = 1.5, color = all_forms[form])
                ax.text(0.1, top , form, horizontalalignment = 'center', verticalalignment = 'bottom', color = all_forms[form], fontsize = 10)
    
    # Azimuth and angle plots
    
    ax11 = axis[0].twiny()
    ax11.plot(well.AZIMUTH, well.index, color = 'blue')
    ax11.spines['top'].set_position(('outward', 0))
    ax11.set_xlim(0, 360)
    ax11.set_xlabel('AZIMUTH[%s]' %las.curves['AZIMUTH'].unit, color = 'blue')    
    ax11.tick_params(axis = 'x', colors = 'blue')
    ax11.set_xticks(np.arange(0, 361, 90))
    ax11.set_xticklabels(['0', '', '180', '', '360'])
    
    ax11.grid(True)

    ax12 = axis[0].twiny()
    ax12.plot(well.ANGLE, well.index, color = 'red')
    ax12.spines['top'].set_position(('outward', 40))   
    ax12.set_xlim(0, 90)
    ax12.set_xlabel('ANGLE[%s]' %las.curves['ANGLE'].unit, color = 'red')    
    ax12.tick_params(axis = 'x', colors = 'red')
    ax12.set_xticks(np.arange(0, 91, 45))
    ax12.set_xticklabels(['0', '45', '90'])
 
    # Gamma ray plot
    
    ax21 = axis[1].twiny()
    ax21.plot(well.GR, well.index, color = 'green')
    ax21.spines['top'].set_position(('outward', 0)) 
    ax21.set_xlim(0, 150)
    ax21.set_xlabel('GR[%s]' %las.curves['GR'].unit, color = 'green')    
    ax21.tick_params(axis = 'x', colors = 'green')
    ax21.set_xticks(np.arange(0, 151, 30))
    ax21.set_xticklabels(['0', '', '', '', '','150'])
    
    ax21.grid(True)
    
    # Resisitivity plots
    
    ax31 = axis[2].twiny()
    ax31.set_xscale('log')
    ax31.plot(well.RT, well.index, color = 'red')
    ax31.spines['top'].set_position(('outward', 0))
    ax31.set_xlim(0.2, 2000)
    ax31.set_xlabel('RT[%s]' %las.curves['RT'].unit, color = 'red')    
    ax31.tick_params(axis = 'x', colors = 'red')
    ax31.set_xticks([0.2, 2, 20, 200, 2000])
    ax31.set_xticklabels(['0.2', '', '', '', '2000'])
    
    ax31.grid(True)

    ax32 = axis[2].twiny()
    ax32.set_xscale('log')
    ax32.plot(well.MSFL, well.index, color = 'black')
    ax32.spines['top'].set_position(('outward', 40))   
    ax32.set_xlim(0.2, 2000)
    ax32.set_xlabel('MSFL[%s]' %las.curves['MSFL'].unit, color = 'black')    
    ax32.tick_params(axis = 'x', colors = 'black')
    ax32.set_xticks([0.2, 2, 20, 200, 2000])
    ax32.set_xticklabels(['0.2', '', '', '', '2000'])
    
    # Density and neutron porosity plots
    
    ax41 = axis[3].twiny()
    ax41.plot(well.RHOB_MRG, well.index, color = 'red')
    ax41.spines['top'].set_position(('outward', 0))
    ax41.set_xlim(1.95, 2.95)
    ax41.set_xlabel('RHOB_MRG[%s]' %las.curves['RHOB_MRG'].unit, color = 'red')    
    ax41.tick_params(axis = 'x', colors = 'red')
    ax41.set_xticks(np.arange(1.95, 2.96, 0.2))
    ax41.set_xticklabels(['1.95', '', '', '', '', '2.95'])
    
    ax41.grid(True)

    ax42 = axis[3].twiny()
    ax42.plot(well.NPHI_MRG, well.index, color = 'blue')
    ax42.spines['top'].set_position(('outward', 40))   
    ax42.set_xlim(0.45, -0.15)
    ax42.set_xlabel('NPHI_MRG[%s]' %las.curves['NPHI_MRG'].unit, color = 'blue')    
    ax42.tick_params(axis = 'x', colors = 'blue')
    ax42.set_xticks(np.arange(0.45, -0.16, -0.12))
    ax42.set_xticklabels(['0.45', '', '', '', '', '-0.15'])
    
    # P_Sonic and S_Sonic plots
    
    ax51 = axis[4].twiny()
    ax51.plot(well.DTC_MRG, well.index, color = 'blue')
    ax51.spines['top'].set_position(('outward', 0))
    ax51.set_xlim(140, 40)
    ax51.set_xlabel('DTC_MRG[%s]' %las.curves['DTC_MRG'].unit, color = 'blue')    
    ax51.tick_params(axis = 'x', colors = 'blue')
    ax51.set_xticks(np.arange(140, 39, -20))
    ax51.set_xticklabels(['140', '', '', '', '', '40'])

    
    ax51.grid(True)

    ax52 = axis[4].twiny()
    ax52.plot(well.DTS_MRG, well.index, color = 'red')
    ax52.spines['top'].set_position(('outward', 40))   
    ax52.set_xlim(340, 40)
    ax52.set_xlabel('DTS_MRG[%s]' %las.curves['DTS_MRG'].unit, color = 'red')    
    ax52.tick_params(axis = 'x', colors = 'red')
    ax52.set_xticks(np.arange(340, 39, -60))
    ax52.set_xticklabels(['340', '', '', '', '', '40'])

    
    # Bad hole flag plots
    
    ax61 = axis[5].twiny()
    ax61.plot(well.BHF, well.index, color = 'red')
    ax61.fill_betweenx(well.index, 0, well.BHF, color = 'red', label = 'Bad hole')
    ax61.spines['top'].set_position(('outward', 0))
    ax61.set_xlabel('BHF', color = 'red')    
    ax61.tick_params(axis = 'x', colors = 'red')
    ax61.set_xticks([0, 1])
    ax61.set_xticklabels(['GOOD', 'BAD'])
    
    ax61.grid(True)
    
    fig.tight_layout()

    # Save files

    logs_folder = 'LQC_Composite_Logs'
    logs_path = os.path.join(sav_path, logs_folder)

    if not os.path.isdir(logs_path):
        os.makedirs(logs_path)

    plt.savefig(os.path.join(logs_path, logs_name), dpi = 200, format = 'png', bbox_inches = "tight")

    plt.show()

# Plot available curves

for las, well, tvd_top in zip(lases, wells, tvd_tops):
    logs_name = 'LQC_' + las.well['WELL'].value + '_Composite_Logs.png'
    composite_logs(las, well, tvd_top, all_forms, logs_name)

In [ ]:
# Function for all well data visualization in composite log plots 

def all_composite_logs(lases, wells, tvd_tops, all_forms, all_logs_name):
    """
    Plot the curves from all wells in composite logs
    lases = las files (.las) of the well data
    wells = well logging data in pandas data frame in TVD depth with alias applied.
    tvd_tops = formation top data in pandas data frame (MD depth is converted to TVD.)
    all_forms = list of all formation names with color code in dictionary format
    all_logs_name = name of saved figure.
    """
    # Create figure and subplots
    
    fig, axis = plt.subplots(nrows = 1, ncols = 6*len(wells), figsize = (50,20), sharey = True)

    top_depth = max([well.index.min() for well in wells])
    bottom_depth = max([well.index.max() for well in wells])

    axis_groups = [list(group) for group in np.array_split(axis, len(wells))]

    for las, well, tvd_top, axis_group in zip(lases, wells, tvd_tops, axis_groups):

        # General setting for all axis
        
        for ax in axis_group:
            ax.set_ylim(top_depth, bottom_depth)
            ax.invert_yaxis()
            ax.minorticks_on() #Scale axis
            ax.get_xaxis().set_visible(False) 
            ax.grid(which = 'major', linestyle = '-', linewidth = '0.5', color = 'green')
            ax.grid(which = 'minor', linestyle = ':', linewidth = '0.5', color = 'blue')
            
            # Plot formations

            for top, form in zip(tvd_top.Top_TVD, tvd_top.Formations):
                if ((top >= well.index.min()) and (top <= well.index.max())):
                    ax.axhline(y = top, linewidth = 1.5, color = all_forms[form])
                    ax.text(0.1, top , form, horizontalalignment = 'center', verticalalignment = 'bottom', color = all_forms[form], fontsize = 10)
        
        # Azimuth and angle plots
        
        ax11 = axis_group[0].twiny()
        ax11.plot(well.AZIMUTH, well.index, color = 'blue')
        ax11.spines['top'].set_position(('outward', 0))
        ax11.set_xlim(0, 360)
        ax11.set_xlabel('AZIMUTH[%s]' %las.curves['AZIMUTH'].unit, color = 'blue')    
        ax11.tick_params(axis = 'x', colors = 'blue')
        ax11.set_xticks(np.arange(0, 361, 90))
        ax11.set_xticklabels(['0', '', '180', '', '360'])
        
        ax11.grid(True)

        ax12 = axis_group[0].twiny()
        ax12.plot(well.ANGLE, well.index, color = 'red')
        ax12.spines['top'].set_position(('outward', 40))   
        ax12.set_xlim(0, 90)
        ax12.set_xlabel('ANGLE[%s]' %las.curves['ANGLE'].unit, color = 'red')    
        ax12.tick_params(axis = 'x', colors = 'red')
        ax12.set_xticks(np.arange(0, 91, 45))
        ax12.set_xticklabels(['0', '45', '90'])
    
        # Gamma ray plot
        
        ax21 = axis_group[1].twiny()
        ax21.plot(well.GR, well.index, color = 'green')
        ax21.spines['top'].set_position(('outward', 0)) 
        ax21.set_xlim(0, 150)
        ax21.set_xlabel('GR[%s]' %las.curves['GR'].unit, color = 'green')    
        ax21.tick_params(axis = 'x', colors = 'green')
        ax21.set_xticks(np.arange(0, 151, 30))
        ax21.set_xticklabels(['0', '', '', '', '','150'])
        
        ax21.grid(True)
        
        # Resisitivity plots
        
        ax31 = axis_group[2].twiny()
        ax31.set_xscale('log')
        ax31.plot(well.RT, well.index, color = 'red')
        ax31.spines['top'].set_position(('outward', 0))
        ax31.set_xlim(0.2, 2000)
        ax31.set_xlabel('RT[%s]' %las.curves['RT'].unit, color = 'red')    
        ax31.tick_params(axis = 'x', colors = 'red')
        ax31.set_xticks([0.2, 2, 20, 200, 2000])
        ax31.set_xticklabels(['0.2', '', '', '', '2000'])
        
        ax31.grid(True)
        ax31.set_title(las.well['WELL'].value, fontsize = 20, y = 1.06)

        ax32 = axis_group[2].twiny()
        ax32.set_xscale('log')
        ax32.plot(well.MSFL, well.index, color = 'black')
        ax32.spines['top'].set_position(('outward', 40))   
        ax32.set_xlim(0.2, 2000)
        ax32.set_xlabel('MSFL[%s]' %las.curves['MSFL'].unit, color = 'black')    
        ax32.tick_params(axis = 'x', colors = 'black')
        ax32.set_xticks([0.2, 2, 20, 200, 2000])
        ax32.set_xticklabels(['0.2', '', '', '', '2000'])
        
        # Density and neutron porosity plots
        
        ax41 = axis_group[3].twiny()
        ax41.plot(well.RHOB_MRG, well.index, color = 'red')
        ax41.spines['top'].set_position(('outward', 0))
        ax41.set_xlim(1.95, 2.95)
        ax41.set_xlabel('RHOB_MRG[%s]' %las.curves['RHOB_MRG'].unit, color = 'red')    
        ax41.tick_params(axis = 'x', colors = 'red')
        ax41.set_xticks(np.arange(1.95, 2.96, 0.2))
        ax41.set_xticklabels(['1.95', '', '', '', '', '2.95'])
        
        ax41.grid(True)

        ax42 = axis_group[3].twiny()
        ax42.plot(well.NPHI_MRG, well.index, color = 'blue')
        ax42.spines['top'].set_position(('outward', 40))   
        ax42.set_xlim(0.45, -0.15)
        ax42.set_xlabel('NPHI_MRG[%s]' %las.curves['NPHI_MRG'].unit, color = 'blue')    
        ax42.tick_params(axis = 'x', colors = 'blue')
        ax42.set_xticks(np.arange(0.45, -0.16, -0.12))
        ax42.set_xticklabels(['0.45', '', '', '', '', '-0.15'])
        
        # P_Sonic and S_Sonic plots
        
        ax51 = axis_group[4].twiny()
        ax51.plot(well.DTC_MRG, well.index, color = 'blue')
        ax51.spines['top'].set_position(('outward', 0))
        ax51.set_xlim(140, 40)
        ax51.set_xlabel('DTC_MRG[%s]' %las.curves['DTC_MRG'].unit, color = 'blue')    
        ax51.tick_params(axis = 'x', colors = 'blue')
        ax51.set_xticks(np.arange(140, 39, -20))
        ax51.set_xticklabels(['140', '', '', '', '', '40'])

        
        ax51.grid(True)

        ax52 = axis_group[4].twiny()
        ax52.plot(well.DTS_MRG, well.index, color = 'red')
        ax52.spines['top'].set_position(('outward', 40))   
        ax52.set_xlim(340, 40)
        ax52.set_xlabel('DTS_MRG[%s]' %las.curves['DTS_MRG'].unit, color = 'red')    
        ax52.tick_params(axis = 'x', colors = 'red')
        ax52.set_xticks(np.arange(340, 39, -60))
        ax52.set_xticklabels(['340', '', '', '', '', '40'])

        
        # Bad hole flag plots
        
        ax61 = axis_group[5].twiny()
        ax61.plot(well.BHF, well.index, color = 'red')
        ax61.fill_betweenx(well.index, 0, well.BHF, color = 'red', label = 'Bad hole')
        ax61.spines['top'].set_position(('outward', 0))
        ax61.set_xlabel('BHF', color = 'red')    
        ax61.tick_params(axis = 'x', colors = 'red')
        ax61.set_xticks([0, 1])
        ax61.set_xticklabels(['GOOD', 'BAD'])
        
        ax61.grid(True)
        
    fig.tight_layout()

    # Save files

    logs_folder = 'LQC_Composite_Logs'
    logs_path = os.path.join(sav_path, logs_folder)

    if not os.path.isdir(logs_path):
        os.makedirs(logs_path)

    plt.savefig(os.path.join(logs_path, all_logs_name), dpi = 200, format = 'png', bbox_inches = "tight")

    plt.show()

# Plot available curves

all_logs_name = 'LQC_All_Composite_Logs.png'
all_composite_logs(lases, wells, tvd_tops, all_forms, all_logs_name)

In [ ]:
# Function for export the data to new las file (.las) and comma-separated values file (.csv)

def export_well(las, well, las_name, csv_name):
    """
    This function can export data to new las file (.las) and comma-separated values file (.csv)
    las = las files (.las) of the well data
    well = well logging data in pandas data frame in TVD depth with alias applied.
    """
    # create a new empty las file

    las_file = lasio.LASFile()

    # export well data (data frame format) to empty las file

    las_file.set_data(well)

    # update curve unit and description
    
    for curve_1, curve_2 in zip(las_file.curves, las.curves):
        if curve_1.mnemonic == curve_2.mnemonic:
            curve_1.unit = curve_2.unit
            curve_1.descr = curve_2.descr

    # update las file header

    las_file.well = las.well

    # update special note for las file
    las_file.other = 'This file was written by python code in %s' %datetime.today().strftime('%m-%d-%Y %H:%M:%S')

    # Save las files

    LQC_las_folder = 'LQC_LAS_files'
    LQC_las_path = os.path.join(sav_path, LQC_las_folder)

    if not os.path.isdir(LQC_las_path):
        os.makedirs(LQC_las_path)
    
    las_file.write(os.path.join(LQC_las_path, las_name), version = 2.0)

    # setup header for csv file

    headers = []

    for curve in las.curves:
        header = '%s[%s]' %(curve.mnemonic, curve.unit)
        headers.append(header)

    index = headers.pop(0)

    # Save csv files

    LQC_csv_folder = 'LQC_CSV_files'
    LQC_csv_path = os.path.join(sav_path, LQC_csv_folder)

    if not os.path.isdir(LQC_csv_path):
        os.makedirs(LQC_csv_path)

    well.rename_axis(index).to_csv(os.path.join(LQC_csv_path, csv_name), header = headers)

# Export las and csv files

for las, well in zip(lases, wells):
    las_name = 'LQC_%s_Input.las' %las.well['WELL'].value
    csv_name = 'LQC_%s_Input.csv' %las.well['WELL'].value
    export_well(las, well, las_name, csv_name)
    print('Well data of %s is exported to las and csv files already.' %las.well['WELL'].value)

# 3.Mechanical Stratigraphy

In [ ]:
# Function for defining the lithology (sand/shale) using normalized gamma ray log

def litho_gr(las, well):
    """
    This function can calculate density porosity (DPHI)
    las = las files (.las) of the well data
    well = well logging data in pandas data frame in TVD depth with alias applied.
    """
    # define sand-shale cut off from normalized gamma ray log

    cutoff = 0.4

    well['LITHO'] = pd.cut(well.GR_NORM, bins = [0, cutoff, 1], labels = ['SAND', 'SHALE'])

    well['liplot'] = 1
    well['liplot'].loc[well.LITHO == 'SHALE'] = 0
    well['liplot'].loc[well.LITHO.isna()] = np.nan

    #create figure

    fig, axis = plt.subplots(nrows = 1, ncols = 2, figsize = (6, 15))
    fig.suptitle(las.well['WELL'].value, fontsize = 15, y = 1.0)

    # General setting for all axis
    
    for ax in axis:
        ax.set_ylim(well.index.min(), well.index.max())
        ax.invert_yaxis()
        ax.minorticks_on() #Scale axis
        ax.get_xaxis().set_visible(False)
        ax.grid(which = 'major', linestyle = '-', linewidth = '0.5', color = 'green')
        ax.grid(which = 'minor', linestyle = ':', linewidth = '0.5', color = 'black') 

    # Gamma ray plot

    ax1 = axis[0].twiny()
    ax1.plot(well.GR, well.index, color = 'green')
    ax1.spines['top'].set_position(('outward', 0)) 
    ax1.set_xlim(0, 150)
    ax1.set_xlabel('GR[%s]' %las.curves['GR'].unit, color = 'green')    
    ax1.tick_params(axis = 'x', colors = 'green')
    ax1.set_xticks(np.arange(0, 151, 30))
    ax1.set_xticklabels(['0', '', '', '', '','150'])

    ax1.grid(True)

    # Bad Hole Flag plot

    ax2 = axis[1].twiny()
    ax2.fill_betweenx(well.index, well.liplot, 1, color = 'SaddleBrown', capstyle = 'butt', linewidth = 0.01, label = 'SHALE')
    ax2.fill_betweenx(well.index, 0, well.liplot, color = 'yellow', capstyle = 'butt', linewidth = 0.01, label = 'SAND')
    ax2.spines['top'].set_position(('outward', 0))
    ax2.set_xlim(0, 1)
    ax2.set_xlabel('LITHOLOGY', color = 'gray')
    ax2.tick_params(axis = 'x', colors = 'gray')
    ax2.set_xticks([0, 1])
    ax2.legend(loc = 'upper left')
    
    ax2.grid(True)
    
    fig.tight_layout()
    
    plt.show()

    well.drop(columns = ['liplot'], inplace = True)

    # update las file

    las.append_curve('LITHO', well['LITHO'], unit = 'unitless', descr = 'Lithology', value = '')

    return las, well

# generate sand-shale lithology

for las, well in zip(lases, wells):
    las, well = litho_gr(las, well)

In [ ]:
# Function for calculating volume of clay

def vcl_cal(las, well):
    """
    This function is for calculating the volume of clay (VCL) using two methods;
    - Neutron-Density based on N-D crossplot for shaly sand formation excluding gas bearing formation (low density with low neutron porosity ).
    - Linear Gamma Ray for gas bearing formation (overestimation of Vcl is possible for high uranium shale)
    las = las files (.las) of the well data.
    well = well logging data in pandas data frame in TVD depth with alias applied.
    """
    # input parameters

    RHOB = well.RHOB_MRG.dropna()
    NPHI = well.NPHI_MRG.dropna()

    # matrix and fluid parameters

    RHOBm, NPHIm = 2.65, 0
    RHOBf, NPHIf = 1.9, 0.45

    # shale parameters

    RHOBsh = well.RHOB_MRG.max()
    NPHIsh = well.NPHI_MRG.max()

    # volume of clay computation from Neutron-Density crossplot equation (Bhuyan and Passey, 1994)

    term1 = (RHOBm-RHOBf)*(NPHI-NPHIf) - (RHOB-RHOBf)*(NPHIm-NPHIf)
    term2 = (RHOBm-RHOBf)*(NPHIsh-NPHIf) - (RHOBsh-RHOBf)*(NPHIm-NPHIf)
    well['VCL'] = term1/term2

    # volume of clay from GR

    VCLgr = well.GR_NORM

    # replace volume of clay from GR in gas bearing formation (VCL < 0)

    well.loc[well.VCL < 0, 'VCL'] = VCLgr

    # limit exceeding volume of clay (VCL > 1) to maximum value (VCL = 1)

    well.VCL = well.VCL.clip(0, 1)

    # update las file

    las.append_curve('VCL', well['VCL'], unit = 'V/V', descr = 'Volume of clay', value = '')

    return las, well

# calculate volume of clay

for las, well in zip(lases, wells):
    las, well = vcl_cal(las, well)

In [ ]:
# Function for calculating effective porosity 

def phie_cal(las, well):
    """
    This function is for calculating the effective porosity (PHIE) using neutron-density conbination;
    las = las files (.las) of the well data.
    well = well logging data in pandas data frame in TVD depth with alias applied.
    """
    # input parameters

    RHOB = well.RHOB_MRG.dropna()
    NPHI = well.NPHI_MRG.dropna()

    # matrix and water parameters

    RHOBm, RHOBw = 2.65, 1.0

    # shale parameters

    RHOBsh = well.RHOB_MRG.max()
    NPHIsh = well.NPHI_MRG.max()

    # density porosity computation with shale correction

    DPHI = (RHOBm - well.RHOB_MRG) / (RHOBm - RHOBw)
    DPHIsh = (RHOBm - RHOBsh) / (RHOBm - RHOBw)
    DPHIshcor = DPHI - (well.VCL * DPHIsh)

    # neutron porosity with shale correction

    NPHIshcor = NPHI - (well.VCL * NPHIsh)

    # total porosity

    POR = np.sqrt(((NPHIshcor**2) + (DPHIshcor**2)) / 2)

    # effective porosity

    well['PHIE'] = POR * (1 - well.VCL)

    # update las file

    las.append_curve('PHIE', well['PHIE'], unit = 'V/V', descr = 'Effective Porosity', value = '')

    return las, well

# calculate volume of clay

for las, well in zip(lases, wells):
    las, well = phie_cal(las, well)

In [ ]:
# Function for neutron-density crossplot

def ndplot(las, well, ndplot_name):
    """
    This function is able to built a crossplot of density and neutron porosity.
    las = las files (.las) of the well data
    well = well logging data in pandas data frame in TVD depth with alias applied.
    ndplot_name = the name of saved figure
    """
    # Create figure

    fig = plt.figure(figsize=(15, 10))

    gs = gridspec.GridSpec(ncols = 2, nrows = 1, width_ratios = [3, 1])
    axis1 = fig.add_subplot(gs[0])
    axis2 = fig.add_subplot(gs[1])
    
    # plot neutron porosity and density

    cmap = mpl.cm.jet

    im = axis1.scatter(well.NPHI_MRG, well.RHOB_MRG, c = well.GR_NORM, marker = '.', cmap = cmap)

    # axis1.plot([0, 2.65], [0.45, 1.9], color = 'black')

    axis1.set_xlabel('NPHI[%s]' %las.curves['NPHI_MRG'].unit)
    axis1.set_ylabel('RHOB[%s]' %las.curves['RHOB_MRG'].unit)
    axis1.set_xlim(-.05, .50)
    axis1.set_ylim(3, 1.8)
    axis1.grid(True)
    
    cbar = fig.colorbar(im, ax = axis1)
    cbar.set_label('NORMALIZED GAMMA RAY')

    axis1.set_title('Neutron-Density Crossplot of %s' %las.well['WELL'].value, fontsize = 15, y = 1.02)

    # general setting for log plot

    axis2.set_ylim(well.index.min(), well.index.max())
    axis2.invert_yaxis()
    axis2.minorticks_on() #Scale axis
    axis2.get_xaxis().set_visible(False)
    axis2.grid(which = 'major', linestyle = '-', linewidth = '0.5', color = 'lightgreen')
    axis2.grid(which = 'minor', linestyle = ':', linewidth = '0.5', color = 'black') 

    # plot effective porosity, rock matrix, volume of clay

    ax1 = axis2.twiny()
    ax1.plot(well.VCL, well.index, color = 'lightgreen')
    ax1.spines['top'].set_position(('outward', 0))
    ax1.set_xlim(0, 1)
    ax1.set_xlabel('VCL[%s]' %las.curves['VCL'].unit, color = 'lightgreen')   
    ax1.tick_params(axis = 'x', colors = 'lightgreen')
    ax1.set_xticks(np.arange(0, 1.1, 0.2))
    ax1.set_xticklabels(['0', '', '', '', '','1'])

    ax1.grid(True)

    ax2 = axis2.twiny()
    ax2.plot(well.PHIE, well.index, color = 'lightgray')
    ax2.spines['top'].set_position(('outward', 40))
    ax2.set_xlim(1, 0)
    ax2.set_xlabel('PHIE[%s]' %las.curves['PHIE'].unit, color = 'gray')   
    ax2.tick_params(axis = 'x', colors = 'gray')
    ax2.set_xticks(np.arange(1.0, -0.1, -0.2))
    ax2.set_xticklabels(['1', '', '', '', '','0'])

    ax2.grid(True)

    ax3 = axis2.twiny()
    ax3.set_xlim(0, 1)
    ax3.spines['top'].set_position(('outward',0))
    ax3.fill_betweenx(well.index, 0, well.VCL, color='lightgreen', capstyle = 'butt', linewidth = 0.5, label = 'VCLAY')
    ax3.fill_betweenx(well.index, well.VCL, (1 - well.PHIE), color='orange', capstyle = 'butt', linewidth = 0.5, label = 'MATRIX')
    ax3.fill_betweenx(well.index, (1 - well.PHIE), 1, color='lightgray', capstyle = 'butt', linewidth = 0.5, label = 'POROSITY')
    ax3.set_xticks([0, 1])
    ax3.set_xticklabels(['', ''])
    ax3.legend(loc = 'upper left')

    ax3.grid(True)

    fig.tight_layout()

    # Save files

    ndplot_folder = 'LQC_NDplot'
    ndplot_path = os.path.join(sav_path, ndplot_folder)

    if not os.path.isdir(ndplot_path):
        os.makedirs(ndplot_path)

    plt.savefig(os.path.join(ndplot_path, ndplot_name), dpi = 200, format = 'png', bbox_inches = "tight")

    plt.show()
        
# generate neutron-density crossplot

for las, well in zip(lases, wells):
    ndplot_name = 'LQC_' + las.well['WELL'].value + '_NDplot.png'
    ndplot(las, well, ndplot_name)

In [ ]:
# Function for multiwell neutron-density crossplot

def multi_ndplot(lases, wells, well_names, multi_ndplot_name):
    """
    This function is able to built a crossplot of density and neutron porosity.
    las = las files (.las) of the well data
    well = well logging data in pandas data frame in TVD depth with alias applied.
    well_names = list of well name and its color identity
    multi_ndplot_name = the name of saved figure
    """
    # Create figure

    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,10))
    fig.suptitle('Neutron-Density Crossplot', fontsize= 15, y = 0.98)
    
    # plot neutron porosity and density

    cmap = mpl.cm.jet

    for well, name in zip(wells, well_names):
        NPHI_sand = well.loc[well.LITHO == 'SAND', 'NPHI_MRG']
        RHOB_sand = well.loc[well.LITHO == 'SAND', 'RHOB_MRG']
        NPHI_shale = well.loc[well.LITHO == 'SHALE', 'NPHI_MRG']
        RHOB_shale = well.loc[well.LITHO == 'SHALE', 'RHOB_MRG']

        ax.scatter(NPHI_sand, RHOB_sand, c = well_names[name], alpha = 0.5, marker = '.', label = 'SAND of %s' %name)
        ax.scatter(NPHI_shale, RHOB_shale, c = well_names[name], alpha = 0.5, marker = '+', label = 'SHALE of %s' %name)

    ax.set_xlabel('NPHI[%s]' %las.curves['NPHI_MRG'].unit)
    ax.set_ylabel('RHOB[%s]' %las.curves['RHOB_MRG'].unit)
    ax.set_xlim(-.05, .50)
    ax.set_ylim(3, 1.8)
    ax.grid(True)
    ax.legend(loc = 'upper left')

    fig.tight_layout()

    # Save files

    ndplot_folder = 'LQC_NDplot'
    ndplot_path = os.path.join(sav_path, ndplot_folder)

    if not os.path.isdir(ndplot_path):
        os.makedirs(ndplot_path)

    plt.savefig(os.path.join(ndplot_path, multi_ndplot_name), dpi = 200, format = 'png', bbox_inches = "tight")

    plt.show()
        
# generate neutron-density crossplot for all wells

multi_ndplot_name = 'LQC_NDcrossplot.png'
multi_ndplot(lases, wells, well_names, multi_ndplot_name)

# 4.Overburden Stress

In [ ]:
# Function for density extrapolation

def den_extra_eq(X, A0, alpha):
    """
    This function is density extrapolation equation.
    RHOml = density at ground level or sea floor
    TVD = true vertival depth (air gap was removed)
    A0 = fitting parameter 1
    alpha = fitting parameter 2
    """
    # independent variables

    RHOml, TVD = X

    # density extrapolation equation

    RHOex = RHOml + (A0 * (TVD**alpha))

    return RHOex

# Function for fitting a density extrapolation curve

def den_extra(las, well, RHOml, point, surface):
    """
    This function is able to fit a density extrapolation curve and create a extrapolated density.
    las = las files (.las) of the well data
    well = well logging data in pandas data frame in TVD depth with alias applied.
    RHOml = density at ground level or sea floor
    surface = position of ground or sea floor
    """
    # input parameters

    RHOB = well.loc[well.index > point, 'RHOB_MRG']

    A = (RHOml, surface)
    B = (RHOB.iloc[0], RHOB.index[0])
    C = (RHOB.iloc[-1], RHOB.index[-1])

    RHOBex = (A[0], B[0], C[0])
    TVDs = (A[1], B[1], C[1])
    RHOs = (RHOml, RHOml, RHOml)
    X = (RHOs, TVDs)

    popt, pcov = curve_fit(den_extra_eq, X, RHOBex)

    well['RHOB_EX'] = RHOml + (popt[0] * (well.index**popt[1]))

    # update las file

    las.append_curve('RHOB_EX', well['RHOB_EX'], unit = las.curves['RHOB_MRG'].unit, descr = 'Extrapolated density', value = '')

    #create figure

    fig, axis = plt.subplots(nrows = 1, ncols = 1, figsize = (4, 10))
    fig.suptitle(las.well['WELL'].value, fontsize = 15, y = 1.0)

    # General setting for all axis
    
    axis.set_ylim(well.index.min(), well.index.max())
    axis.invert_yaxis()
    axis.minorticks_on() #Scale axis
    axis.get_xaxis().set_visible(False)
    axis.grid(which = 'major', linestyle = '-', linewidth = '0.5', color = 'green')
    axis.grid(which = 'minor', linestyle = ':', linewidth = '0.5', color = 'black')

    # Density and extrapolated density

    ax1 = axis.twiny()
    ax1.plot(well.RHOB_MRG, well.index, color = 'red')
    ax1.spines['top'].set_position(('outward', 0))
    ax1.set_xlim(1.95, 2.95)
    ax1.set_xlabel('RHOB[%s]' %las.curves['RHOB_MRG'].unit, color = 'red')    
    ax1.tick_params(axis = 'x', colors = 'red')
    ax1.set_xticks(np.arange(1.95, 2.96, 0.2))
    ax1.set_xticklabels(['1.95', '', '', '', '', '2.95'])

    ax1.grid(True)

    ax2 = axis.twiny()
    ax2.plot(well.RHOB_EX, well.index, color = 'black')
    ax2.spines['top'].set_position(('outward', 40))
    ax2.set_xlim(1.95, 2.95)
    ax2.set_xlabel('RHOB_EX[%s]' %las.curves['RHOB_EX'].unit, color = 'black')    
    ax2.tick_params(axis = 'x', colors = 'black')
    ax2.set_xticks(np.arange(1.95, 2.96, 0.2))
    ax2.set_xticklabels(['1.95', '', '', '', '', '2.95'])
    
    ax2.grid(True)
    
    fig.tight_layout()
    
    plt.show()

    return las, well

# generate extrapolated density curve

for las, well, RHOml, point in zip(lases, wells, RHOmls, starting_points):

    surface = 0

    las, well = den_extra(las, well, RHOml, point, surface)

In [ ]:
wells[2]

In [ ]:
lases[2].curves

In [ ]:
for las in lases:
    del las.curves['RHOB_EX']

for well in wells:
    well.drop(columns = ['RHOB_EX'], inplace = True)

In [ ]:
len(np.arange(0, 920, 0.191))

In [ ]:
starting_points[0]

In [ ]:
test = wells[0].loc[wells[0].index > starting_points[0], 'RHOB_MRG']
test

In [ ]:
test.iloc[0]

In [ ]:
test.index[0]

In [ ]:
test.iloc[-1]

In [ ]:
TVDex = pd.DataFrame(np.arange(0, welltest.index.min(), 0.191), columns = ['TVD'])
TVDex

In [ ]:
welltest.index.min()

In [ ]:
851.672385 - 851.669

In [ ]:
TVD = pd.Series(test.index)
TVD

In [ ]:
pd.concat([TVDex, TVD])

In [ ]:
    # Merge deviation file with well data
    
    df_las = df_las.reset_index()
    df_las = pd.concat([dev[['MD', 'AZIMUTH', 'ANGLE', 'TVD']], df_las]).sort_values(by = ['MD']).reset_index(drop = True)
    
    # Insert true vertical depth using linear interpolation
    
    for col in df_las[['AZIMUTH', 'ANGLE', 'TVD']].columns:
        df_las[col] = df_las[col].interpolate(method = 'linear', limit_area = 'inside')
        
    # Set true vertical depth as file indices
        
    df_las = df_las.dropna(subset = ['TVD']).set_index('TVD')
    df_las = df_las.drop(list(dev['TVD']))

In [ ]:
welltest = wells[0].copy()

In [ ]:
welltest.reset_index(inplace = True)

In [ ]:
welltest

In [ ]:
welltest = pd.concat([TVDex, welltest]).sort_values(by = ['TVD']).reset_index(drop = True)
welltest.set_index('TVD')

In [ ]:
26921 + 4460 